# Исследование надёжности заёмщиков

## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [7]:
# Импортируем pandas и изучим общую информацию.
import pandas as pd


df = pd.read_csv('/datasets/data.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Изучая общую информацию по предоставленным данным, замечаем проблемы со столбцами *'days_employed'* и *'total_income'* — по множеству клиентов у банка нет информации о трудовом стаже и доходах. Скорее всего, эти пропуски появились по причине того, что клиенты не всегда указывают эти данные. 
Число пропущенных значений слишком большое (больше 10%), поэтому без риска потерять выборку удалить строки с пропущенными значениями не получится. Будем заполнять пропуска через медиану, соответствующую каждому типу занятости.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [8]:
# Создаём таблицы, где будут храниться медианы для каждого типа занятости,
# и список уникальных значений типов занятости, чтобы использовать их в цикле.
income_median = df.groupby('income_type')['total_income'].median()
days_employed_median = df.groupby('income_type')['days_employed'].median()
income_type_unique = df['income_type'].unique()

# Заполняем пропущенные значения медианой, соответствующей каждому типу занятости.
for unique in income_type_unique:
    df.loc[df.loc[:,'income_type'] == unique, 'total_income'] = df[df['income_type'] == unique]['total_income'].fillna(income_median[unique])
    df.loc[df.loc[:,'income_type'] == unique, 'days_employed'] = df[df['income_type'] == unique]['days_employed'].fillna(days_employed_median[unique])
    
# Проверим, остались ли пропущенные значения.
print(f"Сумма пропущенных значений: {df['total_income'].isnull().sum() + df['days_employed'].isnull().sum()}")

Сумма пропущенных значений: 0


### Вывод

Подставить медиану, общую для всех типов занятости, было бы неправильно, так как не учитывалась бы разница между, например, студентом и предпринимателем, данные бы исказились. Пришлось искать медиану отдельно для каждого типа занятости, чтобы подставить вместо пропусков более приближенные к реальности значения.
Мы продолжим работать с данными столбцами, и следующим шагом будет замена типа данных с float на int (для облегчения расчётов, и по логике в стаж в днях не должна записываться "доля дня")  и приведение данных в столбце *'days_employed'* к положительному значению, так как стаж не может быть отрицательным.

### Замена типа данных

In [9]:
# Заменим тип данных в столбцах 'days_employed' и 'total_income' с float на int
# и приведём к положительному значению.
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = abs(df['days_employed'].astype('int'))
df['children'] = abs(df['children'])

# Проверим, получилось ли заменить тип данных и сделать значения положительными.
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Успешно получилось привести данные в столбцах в порядок, сделав их целочисленными с помощью метода *astype()*, тем самым убрав ненужные знаки после запятой, а также сделать значения стажа положительными при помощи метода abs(), приводящего число к абсолютному значению, чтобы число, изменённое с помощью этого метода, всегда получалось положительным. 
Далее, нужно выяснить, есть ли в таблице дубликаты и если есть, удалить.

### Обработка дубликатов

In [10]:
# В столбце 'education' у в теории одинаковых значений разный регистр,
# приведём все символы в нём к нижнему регистру.
df['education'] = df['education'].str.lower()

# Проверим уникальные значения стоблца.
print(df['education'].unique())

# Удалим дубликаты и зададим новую индексацию, а потом проверим, удалились ли дубликаты.
print(f"Количество дубликатов: {df.duplicated().sum()}")
df = df.drop_duplicates().reset_index(drop = True)
if df.duplicated().sum() == 0:
    print("Дубликатов больше нет!")
else:
    print("Дубликаты не были удалены.")

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
Количество дубликатов: 71
Дубликатов больше нет!


### Вывод

Приведя символы в столбце *education* к нижнему регистру с помощью str.lower(), мы привели данные в порядок, убрав одинаковые значения с разным регистром и облегчили тем самым поиск дубликатов. Самих дубликатов оказалось мало относительно всей таблицы, и мы спокойно их удалили. Скорее всего, они появились при повторном обращении клиента в банк и повторном указании своих данных, на что сотрудники не обратили внимания.

### Лемматизация

In [11]:
# Импортируем pymystem3 для осуществления лемматизации.
from pymystem3 import Mystem
m = Mystem()

# Переводим тип данных из object в string для корректной обработки.
df['purpose'] = df['purpose'].astype('str')


def lemmas(row):
    '''
    Лемматизирует данные в столбце 
    и возвращает список лемматизированных слов.
    '''
    text = row['purpose']
    output = ' '.join(m.lemmatize(text))
    return output

  
# Создаём новый столбец с лемматизированными словами, применяя ранее написанную функцию.
df['lemmas'] = df.apply(lemmas, axis=1)

df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n


С помощью написаной функции, мы смогли получить список лемматизированных слов из столбца *'purpose'* и записать их в новый стобцей *'lemmas.'*
Следующим шагом будем категоризация данных по этим четырём категориям, а так же категоризация по уровню дохода.

### Категоризация данных

In [12]:
# Создаём "словарь" для образования,
# чтобы расчистить место в основной таблице для более лёгкой обработки.
edu_dict = df[['education_id', 'education']].drop_duplicates().reset_index(drop=True)

# Удаляем эти столбцы из основной таблицы.
df = df.drop(['education', 'education_id'], axis='columns')

def categorize_main_purpose(row):
    """
    Возвращает главную цель кредита в зависимости от наличия слов в столбце 'lemmas',
    используя правила:
    - 'Оплата недвижимости' при наличии слов 'жилье' или 'недвижимость'
    - 'Оплата автомобиля' при наличии слова 'автомобиль'
    - 'Оплата образования' при наличии слова 'образование'
    - 'Оплата свадьбы' при наличии слова 'свадьба'
    """
    
    lemmas = row['lemmas']
    if 'жилье' in lemmas:
        return 'Оплата недвижимости'
    
    if 'недвижимость' in lemmas:
        return 'Оплата недвижимости'

    if 'автомобиль' in lemmas:
        return 'Оплата автомобиля'
        
    if 'образование' in lemmas:
        return 'Оплата образования'
    
    if 'свадьба' in lemmas:
        return 'Оплата свадьбы'
        
def categorize_income_size(row):
    """
    Возвращает уровень дохода в зависимости от месячного дохода в столбце total_income,
    изпользуя правила:
    - 'Низкий доход' при доходе до 45000 рублей.
    - 'Средний доход' при доходе до 80000 рублей.
    - 'Средне-высокий доход' при доходе до 150000 рублей.
    - 'Высокий доход' при доходе до 250000 рублей.
    - 'Очень высокий доход' при доходе до 500000 рублей.
    - 'Сверхвысокий доход' при доходе от 500000 рублей.
    """
    
    income = row['total_income']
    if income < 45000:
        return 'Низкий доход'
    
    if income < 80000:
        return 'Средний доход'
    
    if income < 150000:
        return 'Средне-высокий доход'
    
    if income < 250000:
        return 'Высокий доход'
    
    if income < 500000:
        return 'Очень высокий доход'
    
    if income > 500000:
        return 'Сверхвысокий доход'

        
# Применим функцию для создания нового столбца 'main_purpose'.  
df['main_purpose'] = df.apply(categorize_main_purpose, axis=1)
df['income_size'] = df.apply(categorize_income_size, axis=1)

df.head(10)

,children,days_employed,dob_years,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,main_purpose,income_size
0,1,8437,42,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,Оплата недвижимости,Очень высокий доход
1,1,4024,36,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,Оплата автомобиля,Средне-высокий доход
2,0,5623,33,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,Оплата недвижимости,Средне-высокий доход
3,3,4124,32,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,Оплата образования,Очень высокий доход
4,0,340266,53,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,Оплата свадьбы,Высокий доход
5,0,926,27,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n,Оплата недвижимости,Очень высокий доход
6,0,2879,43,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n,Оплата недвижимости,Высокий доход
7,0,152,50,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n,Оплата образования,Средне-высокий доход
8,2,6929,35,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n,Оплата свадьбы,Средне-высокий доход
9,0,2188,41,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n,Оплата недвижимости,Средне-высокий доход


### Вывод

С помощью лемматизации ранее можно было заметить, что люди берут кредиты для четырёх целей: отплаты недвижимости, автомобиля, образования и свадьбы. Поэтому, было решено создать новый столбец *'main_purpose'* для категоризации данных по этим четырём главным целям. Обобщение же по уровню дохода показало, что достаточно много людей с высоким и выше доходом тоже берут кредиты.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [13]:
# Создаём сводную таблицу. 
child_pivot_table = df.pivot_table(index = 'children', columns = 'debt', values = 'dob_years', aggfunc = 'count')

# Выяснилось, что среди людей с пятью детьми не нашлось тех, кто не отдавал кредит вовремя, поэтому
# в столбце возвращалось NaN. Пришлось использовать fillna(), чтобы всё-таки добавить нулевое значение
# для корректной работы.
child_pivot_table = child_pivot_table.fillna(0)

# Создаём стобцы: с общим количеством кредитов и процентом не возвращённых вовремя кредитов.
child_pivot_table['debt_all'] = child_pivot_table[0] + child_pivot_table[1]
child_pivot_table['debt_percent'] = child_pivot_table[1] / child_pivot_table['debt_all']

# Форматируем и сортируем по убыванию столбец debt_percent.
child_pivot_table = child_pivot_table.sort_values(by='debt_percent', ascending=False)
child_pivot_table = child_pivot_table.style.format({'debt_percent': '{:,.2%}'.format})

child_pivot_table

debt,0,1,debt_all,debt_percent
children,,,,
20,68,8,76,10.53%
4,37,4,41,9.76%
2,1858,194,2052,9.45%
1,4410,445,4855,9.17%
3,303,27,330,8.18%
0,13028,1063,14091,7.54%
5,9,0,9,0.00%


### Вывод

Чаще всего возвращают кредиты люди, не имеющие детей и имеющие 5 детей, но в последнем случае слишком маленькая выборка. Далее, как можно видеть, в среднем клиенты, имеющие детей, чаще не возвращают кредит в срок, и это значение может увеличиваться в зависимости от количества детей. Но клиенты с тремя детьми чаще возвращают кредиты в срок, чем клиенты с другим количеством детей (1, 2, 4, 20).
Вывод: зависимость между наличием детей и возвратом кредита в срок имеет место быть, клиенты без детей чаще возвращают кредит в срок, чем клиенты с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [14]:
# Создаём сводную таблицу. 
family_pivot_table = df.pivot_table(index = 'family_status', columns = 'debt', values = 'dob_years', aggfunc = 'count')

# Создаём стобцы: с общим количеством кредитов и процентом не возвращённых вовремя кредитов.
family_pivot_table['debt_all'] = family_pivot_table[0] + family_pivot_table[1]
family_pivot_table['debt_percent'] = family_pivot_table[1] / family_pivot_table['debt_all']

# Форматируем и сортируем по убыванию столбец debt_percent.
family_pivot_table = family_pivot_table.sort_values(by='debt_percent', ascending=False)
family_pivot_table = family_pivot_table.style.format({'debt_percent': '{:,.2%}'.format})

family_pivot_table

debt,0,1,debt_all,debt_percent
family_status,,,,
Не женат / не замужем,2536,274,2810,9.75%
гражданский брак,3763,388,4151,9.35%
женат / замужем,11408,931,12339,7.55%
в разводе,1110,85,1195,7.11%
вдовец / вдова,896,63,959,6.57%


### Вывод

Чаще всего не возвращают кредиты в срок люди без пары или живущие в гражданском браке (сожительство). Процент возврата в срок выше у людей, у которых есть или был зарегистрированный брак. 
Вывод: зависимость имеет место быть.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [15]:
# Создаём сводную таблицу. 
income_pivot_table = df.pivot_table(index = 'income_size', columns = 'debt', values = 'dob_years', aggfunc = 'count')

# Создаём стобцы: с общим количеством кредитов и процентом не возвращённых вовремя кредитов.
income_pivot_table['debt_all'] = income_pivot_table[0] + income_pivot_table[1]
income_pivot_table['debt_percent'] = income_pivot_table[1] / income_pivot_table['debt_all']

# Форматируем и сортируем по убыванию столбец debt_percent.
income_pivot_table = income_pivot_table.sort_values(by='debt_percent', ascending=False)
income_pivot_table = income_pivot_table.style.format({'debt_percent': '{:,.2%}'.format})

income_pivot_table

debt,0,1,debt_all,debt_percent
income_size,,,,
Средне-высокий доход,8543,804,9347,8.60%
Высокий доход,6449,569,7018,8.11%
Средний доход,1899,160,2059,7.77%
Очень высокий доход,2411,180,2591,6.95%
Низкий доход,203,14,217,6.45%
Сверхвысокий доход,208,14,222,6.31%


### Вывод

Как ни странно, оказалось, что чаще всего возвращают кредиты в срок, помимо людей со сверхвысоким доходом, именно клиеты с низким доходом и средним доходом. Люди со средним и низким доходом в возвращают в срок в целом чаще, чем у большинства людей с уровнем дохода выше их (средне-высокий, высокий доход). 

- Как разные цели кредита влияют на его возврат в срок?

In [16]:
# Создаём сводную таблицу. 
purpose_pivot_table = df.pivot_table(index = ['main_purpose'], columns = 'debt', values = 'dob_years', aggfunc = 'count')

# Создаём стобцы: с общим количеством кредитов и процентом не возвращённых вовремя кредитов.
purpose_pivot_table['debt_all'] = purpose_pivot_table[0] + purpose_pivot_table[1]
purpose_pivot_table['debt_percent'] = purpose_pivot_table[1] / purpose_pivot_table['debt_all']

# Форматируем и сортируем по убыванию столбец debt_percent.
purpose_pivot_table = purpose_pivot_table.sort_values(by='debt_percent', ascending=False)
purpose_pivot_table = purpose_pivot_table.style.format({'debt_percent': '{:,.2%}'.format})
purpose_pivot_table

debt,0,1,debt_all,debt_percent
main_purpose,,,,
Оплата автомобиля,3903,403,4306,9.36%
Оплата образования,3643,370,4013,9.22%
Оплата свадьбы,2138,186,2324,8.00%
Оплата недвижимости,10029,782,10811,7.23%


### Вывод

Чаще всего просрочивают оплату кредита, который давался на покупку автомобиля или оплаты образования, реже всего просрочивают оплату кредита по недвижимости и на оплату свадьбы. 

### Шаг 4. Общий вывод

Выяснилось, что семейное положение и количество детей всё-таки влияют на платёжеспособность клиента. Чаще всего в срок возвращают клиенты без детей, а также клиенты, состоящие в браке или когда-либо в нём состоявшие (разведены или супруг/супруга скончались).
Также выяснилось, что клиенты с низким доходом вносят оплату вовремя чаще, чем более богатые клиенты, исключая людей со сверхдоходами (больше 500 тысяч рублей), а просрочивают чаще всего платежи по кредиту на автомобиль.